In [6]:
#usage python import_data <state_name>

import array, math, os, psycopg2, random, sys
from shapely.geometry import *
from shapely.wkb import loads

#state_name = sys.argv[0];
state_name = 'ak'

def init_capture(capture_dir):
    # Create capture dir if it doesn't exist
    if not os.path.exists(capture_dir):
        os.makedirs(capture_dir)
        
def state_lookup(state_name):
    found = [item for item in ANSI_CODES if item[1] == state_name]
    if found:
        return found[0][0]
    return -1;

def format_tabblock_url(state):
    # URL for FIPS and GNIS codes file -- https://www.census.gov/geo/reference/docs/state.txt'
    return 'ftp://ftp2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_{0}_tabblock10.zip'.format(state)

def format_lodes7_od_url(st, part, t, year):
    #http://lehd.ces.census.gov/data/lodes/LODES7/LODESTechDoc7.0.pdf
    return 'http://lehd.ces.census.gov/data/lodes/LODES7/{0}/od/{0}_od_{1}_{2}_{3}.csv.gz'.format(st, part, t, year)

def download_file(url, filename):
    command = "wget %s -O %s" % (url, filename)
    print "Downloading %s to %s" % (url, filename)
    !$command

def unzip_tabblock(filename, exdir):
    command = "unzip %s -d %s" % (filename, exdir)
    print "Unzip %s to %s" % (filename, exdir)
    !$command

def gunzip_lodes(filename):
    command = "gunzip %s" % filename
    print "Unzip %s" % filename
    !$command

def create_od_table(state):
    s =(
        "DROP TABLE IF EXISTS %s_od_jt00_2011;\n"
        "CREATE TABLE %s_od_jt00_2011 ( "
        "gid serial NOT NULL, "
        "w_geocode character varying(15), "
        "h_geocode character varying(15), "
        "S000 integer, "
        "SA01 integer, " 
        "SA02 integer, "
        "SA03 integer, "
        "SE01 integer, "
        "SE02 integer, "
        "SE03 integer, "
        "SI01 integer, "
        "SI02 integer, "
        "SI03 integer, "
        "createdate character varying(8));\n"
    ) % (state,state)
    return s

def copy_cvs_to_psql(state,filename):
    s = (
         "COPY %s_od_jt00_2011("
         "w_geocode,h_geocode,S000,SA01,"
         "SA02,SA03,SE01,SE02,SE03,SI01,"
         "SI02,SI03,createdate) FROM "
         "'%s' "
        "DELIMITER ',' CSV HEADER;\n"
         ) % (state, filename)
    return s

ANSI_CODES = [
    ('01', 'al'), ('02', 'ak'), ('04', 'az'), ('05', 'ar'), ('06', 'ca'),
    ('08', 'co'), ('09', 'ct'), ('10', 'de'), ('11', 'dc'), ('12', 'fl'),
    ('13', 'ga'), ('15', 'hi'), ('16', 'id'), ('17', 'il'), ('18', 'in'),
    ('19', 'ia'), ('20', 'ks'), ('21', 'ky'), ('22', 'la'), ('23', 'me'),
    ('24', 'md'), ('25', 'ma'), ('26', 'mi'), ('27', 'mn'), ('28', 'ms'),
    ('29', 'mo'), ('30', 'mt'), ('31', 'ne'), ('32', 'nv'), ('33', 'nh'),
    ('34', 'nj'), ('35', 'nm'), ('36', 'ny'), ('37', 'nc'), ('38', 'nd'),
    ('39', 'oh'), ('40', 'ok'), ('41', 'or'), ('42', 'pa'), ('44', 'ri'),
    ('45', 'sc'), ('46', 'sd'), ('47', 'tn'), ('48', 'tx'), ('49', 'ut'),
    ('50', 'vt'), ('51', 'va'), ('53', 'wa'), ('54', 'wi'), ('55', 'wy'),
    ('56', 'wv')
]

In [2]:
init_capture("capture/tabblock_2010")
capture_dir = "capture/tabblock_2010/"
state_id = state_lookup(state_name);
init_capture(capture_dir)
state_url = format_tabblock_url(state_id)
state_filename = capture_dir + os.path.basename(state_url)
#download_file(state_url, state_filename)
#unzip_tabblock(state_filename, state_filename.split('.')[0])

NameError: name 'init_capture' is not defined

In [3]:
command = "createdb tl_2010_tabblock"
!$command
command = "psql -d tl_2010_tabblock -c 'CREATE EXTENSION postgis;'"
!$command

createdb: database creation failed: ERROR:  database "tl_2010_tabblock" already exists
ERROR:  extension "postgis" already exists


In [ ]:
state_id = state_lookup(state_name);
command = "shp2pgsql -s 4269:4326 capture/tabblock_2010/tl_2010_%s_tabblock10/tl_2010_%s_tabblock10.shp tl_2010_tabblock10 #| psql -q -d tl_2010_tabblock" % (state_id,state_id)
!$command

Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]
SET CLIENT_ENCODING TO UTF8;
SET STANDARD_CONFORMING_STRINGS TO ON;
BEGIN;
CREATE TABLE "tl_2010_tabblock10" (gid serial,
"statefp10" varchar(2),
"countyfp10" varchar(3),
"tractce10" varchar(6),
"blockce10" varchar(4),
"geoid10" varchar(15),
"name10" varchar(10),
"mtfcc10" varchar(5),
"ur10" varchar(1),
"uace10" varchar(5),
"uatyp10" varchar(1),
"funcstat10" varchar(1),
"aland10" float8,
"awater10" float8,
"intptlat10" varchar(11),
"intptlon10" varchar(12));
ALTER TABLE "tl_2010_tabblock10" ADD PRIMARY KEY (gid);
SELECT AddGeometryColumn('','tl_2010_tabblock10','geom','4326','MULTIPOLYGON',2);
INSERT INTO "tl_2010_tabblock10" ("statefp10","countyfp10","tractce10","blockce10","geoid10","name10","mtfcc10","ur10","uace10","uatyp10","funcstat10","aland10","awater10","intptlat10","intptlon10",geom) VALUES ('02','013','000100','2132','020130001002132','Block 2132','G5040','R',NULL,NULL,'S','0','1140917','+56.0119409','-161.0036526',ST_Tra

In [ ]:
command = "psql -d tl_2010_tabblock -c 'CREATE INDEX ON tl_2010_tabblock10 (geoid10);'"
!$command

In [ ]:
command = "psql -d tl_2010_tabblock -c 'CREATE INDEX ON tl_2010_tabblock10 USING GIST (geom);'"
!$command

In [ ]:
capture_dir = "capture/lodes7/{0}/".format(state_name);
init_capture(capture_dir)
state_url = format_lodes7_od_url(state_name, 'main', 'JT00', '2011')
state_filename = capture_dir + os.path.basename(state_url)
if not os.path.isdir('./' + capture_dir):
    download_file(state_url, state_filename) 
    gunzip_lodes(state_filename)


In [ ]:
state_file = "{0}-csv-to-psql.sql".format(state_name)
f = open(state_file, "w")
f.write(create_od_table(state_name))
f.write(copy_cvs_to_psql(state_name, os.path.abspath(os.path.splitext(state_filename)[0])))
f.close()
command = "psql -d tl_2010_tabblock -f {0}".format(state_file)
!$command

command = "psql -d tl_2010_tabblock -c 'CREATE INDEX ON {0}_od_jt00_2011 (gid);'".format(state_name)
!$command

command = "psql -d tl_2010_tabblock -c 'CREATE INDEX ON {0}_od_jt00_2011 (w_geocode);'".format(state_name)
!$command

command = "psql -d tl_2010_tabblock -c 'CREATE INDEX ON {0}_od_jt00_2011 (h_geocode);'".format(state_name)
!$command